In [1]:
import os
import graphviz
# os.environ["PATH"] += os.pathsep + 'C:\Program Files\Graphviz\bin'

In [2]:
sys.path.insert(1,r'F:\Users\Kaggle\bulian')

In [3]:
import os,sys,torch
import pandas as pd

c:\Users\anaconda\envs\BulianAI\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
from bulian import load_demo
tables = load_demo()

In [5]:
tables

{'users':    user_id country gender  age
 0        0      US      M   34
 1        1      UK      F   23
 2        2      ES   None   44
 3        3      UK      M   22
 4        4      US      F   54
 5        5      DE      M   57
 6        6      BG      F   45
 7        7      ES   None   41
 8        8      FR      F   23
 9        9      UK   None   30,
 'sessions':    session_id  user_id  device       os  minutes
 0           0        0  mobile  android       23
 1           1        1  tablet      ios       12
 2           2        2  tablet  android        8
 3           3        3  mobile  android       13
 4           4        4  mobile      ios        9
 5           5        5  mobile  android       32
 6           6        6  mobile      ios        7
 7           7        7  tablet      ios       21
 8           8        8  mobile      ios       29
 9           9        9  tablet      ios       34,
 'transactions':    transaction_id  session_id           timestamp  amount 

In [6]:
from bulian import Metadata

metadata = Metadata()
metadata

Metadata
  root_path: .
  tables: []
  relationships:

In [7]:
metadata.add_table(
    name='users',
    data=tables['users'],
    primary_key='user_id'
)

In [9]:
# metadata.visualize()

In [10]:
metadata.get_table_meta('users')

{'fields': {'user_id': {'type': 'id', 'subtype': 'integer'},
  'country': {'type': 'categorical'},
  'gender': {'type': 'categorical'},
  'age': {'type': 'numerical', 'subtype': 'integer'}},
 'primary_key': 'user_id'}

In [11]:
for name, table in tables.items():
    print(name, table.shape)

users (10, 4)
sessions (10, 5)
transactions (10, 5)


In [12]:
from bulian.utils import display_tables
display_tables(tables)

ImportError: cannot import name 'display_tables' from 'bulian.utils' (F:\Users\Kaggle\bulian\bulian\utils.py)

In [ ]:
from bulian.Tabular.synthesizers import TwinSynthesizer,PrivateTwinSynthesizer
from bulian.metrics import *
from bulian.metrics.reports import *
from bulian.metrics import compute_metrics
from bulian.metrics.single_table import SingleTableMetric
from bulian.metrics.single_table import *

In [ ]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

In [ ]:
discrete_columns = [
    "workclass",
    "education",
    "marital-status",
    "occupation",
    "relationship",
    "race",
    "sex",
    "native-country",
    "income",
]

In [ ]:
torch.cuda.is_available()

In [ ]:
!nvidia-smi

###### 

## Normal API: Non-privately differentiable synthesizer

In [ ]:
data = pd.read_csv(r"F:\Users\Kaggle\bulian\examples\csv\adult.csv")

In [ ]:
data.head()

In [ ]:
synth = TwinSynthesizer(batch_size=200,device='cuda')   ### cpu else cuda

In [ ]:
synth.fit(data=data,epochs=2,discrete_columns=discrete_columns)

In [ ]:
sample = synth.sample(1000)

In [ ]:
metrics = SingleTableMetric.get_subclasses()
numeric_features = ['capital-gain','capital-loss','hours-per-week']
discrete_columns = [
    "workclass",
    "education",
    "marital-status",
    "occupation",
    "relationship",
    "race",
    "sex",
    "native-country",
    "income",
]

In [ ]:
metrics

In [ ]:
compute_metrics(metrics,data, sample)

#### Report with privacy metrics

In [ ]:
get_full_report(data, sample,discrete_columns,numeric_features, key_fields=['age','workclass','education'],sensitive_fields = ['income'])

#### Report without privacy metrics, but includes ML efficacy stuff

In [ ]:
get_full_report(data, sample,discrete_columns,numeric_features,target='income')

#### Report without privacy metrics and without ML efficacy stuff

In [ ]:
get_full_report(data, sample,discrete_columns,numeric_features)

#### Save model to disk

In [ ]:
synth.save('F:/Users/Kaggle/wids/NormalAPI.pth')

#### 

## Adversarial API: Non-privately differentiable synthesizer

In [ ]:
synth = TwinSynthesizer(batch_size=200,device='cpu')   ### cpu else cuda

In [ ]:
synth.fit_adversarial(data=data,epochs=2,discrete_columns=discrete_columns,test_pct=0.33)

In [ ]:
sample = synth.sample_adversarial(data,1000)

In [ ]:
metrics = SingleTableMetric.get_subclasses()
numeric_features = ['capital-gain','capital-loss','hours-per-week']
discrete_columns = [
    "workclass",
    "education",
    "marital-status",
    "occupation",
    "relationship",
    "race",
    "sex",
    "native-country",
    "income",
]

#### Report with privacy metrics

In [ ]:
get_full_report(data, sample,discrete_columns,numeric_features, key_fields=['age','workclass','education'],sensitive_fields = ['income'])

#### Report without privacy metrics, but includes ML efficacy stuff

In [ ]:
get_full_report(data, sample,discrete_columns,numeric_features,target='income')

#### Report without privacy metrics and without ML efficacy stuff

In [ ]:
get_full_report(data, sample,discrete_columns,numeric_features)

#### Save model to disk

In [ ]:
synth.save('F:/Users/Kaggle/wids/AdversarialAPI.pth')

#### 

## Normal API: Privately differentiable synthesizer

In [ ]:
synth = PrivateTwinSynthesizer(epsilon=0.1,batch_size=64,device='cuda')   ### cpu else cuda

In [ ]:
synth.fit(data=data,discrete_columns=discrete_columns,update_epsilon=1)

In [ ]:
sample = synth.sample(1000)

In [ ]:
metrics = SingleTableMetric.get_subclasses()
numeric_features = ['capital-gain','capital-loss','hours-per-week']
discrete_columns = [
    "workclass",
    "education",
    "marital-status",
    "occupation",
    "relationship",
    "race",
    "sex",
    "native-country",
    "income",
]

#### Report with privacy metrics

In [ ]:
get_full_report(data, sample,discrete_columns,numeric_features, key_fields=['age','workclass','education'],sensitive_fields = ['income'])

#### Report without privacy metrics, but includes ML efficacy stuff

In [ ]:
get_full_report(data, sample,discrete_columns,numeric_features,target='income')

#### Report without privacy metrics and without ML efficacy stuff

In [ ]:
get_full_report(data, sample,discrete_columns,numeric_features)

#### Save model to disk

In [ ]:
synth.save('F:/Users/Kaggle/wids/PrivateModelNormalAPI.pth')

#### 

## Adversarial API: Privately differentiable synthesizer

In [ ]:
synth = PrivateTwinSynthesizer(epsilon=0.1,batch_size=64,device='cuda')   ### cpu else cuda

In [ ]:
synth.fit_adversarial(data=data,discrete_columns=discrete_columns,update_epsilon=1)

In [ ]:
sample = synth.sample_adversarial(data,1000)

In [ ]:
metrics = SingleTableMetric.get_subclasses()
numeric_features = ['capital-gain','capital-loss','hours-per-week']
discrete_columns = [
    "workclass",
    "education",
    "marital-status",
    "occupation",
    "relationship",
    "race",
    "sex",
    "native-country",
    "income",
]

#### Report with privacy metrics

In [ ]:
get_full_report(data, sample,discrete_columns,numeric_features, key_fields=['age','workclass','education'],sensitive_fields = ['income'])

#### Report without privacy metrics, but includes ML efficacy stuff

In [ ]:
get_full_report(data, sample,discrete_columns,numeric_features,target='income')

#### Report without privacy metrics and without ML efficacy stuff

In [ ]:
get_full_report(data, sample,discrete_columns,numeric_features)

#### Save model to disk

In [ ]:
synth.save('F:/Users/Kaggle/wids/PrivateModelAdvAPI.pth')

#### 

### Fin ###